In [ ]:
# Import statements
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import sys
from glob import glob
from pymatreader import read_mat
from scipy import signal
from scipy import stats
from scipy.ndimage import uniform_filter1d
import time

#import matlab.engine
# Test out matlab spike detection code
#eng = matlab.engine.start_matlab()
#eng.eval("startup", nargout=0)

# Importing custom python files
import consts
import importlib
importlib.reload(consts)

from open_ephys.analysis import Session

%matplotlib tk

In [ ]:
# Functions

# Get all of the ephys data and section everything across all trials
def read_openephys_data(pathname, start_chan, frame_chan, stim_chan, flicker_chan):
    session = Session(pathname)
    ephys_data = session.recordnodes[0].recordings[0].continuous[0]

    # Extract and find the start times of recording trials
    start_chan_idx = ephys_data.metadata['channel_names'].index(start_chan)
    raw_start_trigger = ephys_data.samples[:, start_chan_idx]

    frame_chan_idx = ephys_data.metadata['channel_names'].index(frame_chan)
    raw_frame_trigger = ephys_data.samples[:, frame_chan_idx]

    stim_chan_idx = ephys_data.metadata['channel_names'].index(stim_chan)
    raw_stim_trigger = ephys_data.samples[:, stim_chan_idx]

    flicker_chan_idx = ephys_data.metadata['channel_names'].index(flicker_chan)
    raw_flicker_trigger = ephys_data.samples[:, flicker_chan_idx]

    # Get all of the rise idxs
    record_start_idx = consts.get_ephys_rise_indices(raw_start_trigger)
    all_frame_idx = consts.get_ephys_rise_indices(raw_frame_trigger)
    all_stim_idx = consts.get_ephys_rise_indices(raw_stim_trigger)
    all_flicker_idx = consts.get_ephys_rise_indices(raw_flicker_trigger)

    # Get the reverse flicker idxs
    #TODO fix this and try to get the reverse indices for the flicker and get the pulse width
    all_flicker_end_idx = consts.get_ephys_rise_indices(raw_flicker_trigger)

    block_frame_timestamps = []
    block_stim_timestamps = []
    block_flicker_timestamps = []

    #DEBUG
    #plt.figure()
    #plt.plot(ephys_data.timestamps, raw_start_trigger)
    #plt.plot(ephys_data.timestamps[record_start_idx], raw_start_trigger[record_start_idx], '|')
    #plt.title('Ephys Debugging')
    #plt.show()

    #plt.figure()
    #plt.plot(ephys_data.timestamps, raw_start_trigger, '-g')
    #plt.plot(ephys_data.timestamps, raw_stim_trigger, '-g')
    #plt.plot(ephys_data.timestamps, raw_flicker_trigger, '-b')

    # Loop through each recording start trigger and parse together all of the trials
    for i in range(record_start_idx.shape[0]):
        start_idx = record_start_idx[i]
        if i == record_start_idx.shape[0] - 1:
            next_idx = float('inf')
        else:
            next_idx = record_start_idx[i + 1]

        # Find all of the camera frame indices for the given block
        frame_idxs = all_frame_idx[(all_frame_idx > start_idx) & (all_frame_idx < next_idx)]
        stim_idxs = all_stim_idx[(all_stim_idx > start_idx) & (all_stim_idx < next_idx)]
        flicker_idxs = all_flicker_idx[(all_flicker_idx > start_idx) & (all_flicker_idx < next_idx)]

        block_frame_timestamps.append(ephys_data.timestamps[frame_idxs])
        block_stim_timestamps.append(ephys_data.timestamps[stim_idxs])
        block_flicker_timestamps.append(ephys_data.timestamps[flicker_idxs])
    
        #plt.plot(ephys_data.timestamps[stim_idxs], raw_stim_trigger[stim_idxs], '|r')
        #plt.plot(ephys_data.timestamps[flicker_idxs], raw_flicker_trigger[flicker_idxs], '|r')
        #plt.plot(ephys_data.timestamps[start_idx], raw_start_trigger[start_idx], '|m')
    #plt.show()

    return block_stim_timestamps, block_frame_timestamps, block_flicker_timestamps

# Get the flicker experiment
def get_param(fname, param):
    # Loop through each element and search for string pattern
    for exp_param in fname.split('_'):
        if param in exp_param:
            return exp_param    
    print('Missing param: ' + param)
    return



In [ ]:
f = os.sep

# Initialize empty dataframe
df = pd.DataFrame()

# Data rootpath is actually in handata3 for now
server_root = '/home/pierfier/handata_server/eng_research_handata3'
data_root_path = f.join([server_root, 'Yangyang_Wang', 'PV_V1_LED_SomArchon']) + f

# Find all folders that have the traces matfile and were flicker experiments
mat_paths = glob(data_root_path + '**/**RawTraces**flicker**DBS**.mat', recursive=True)

# Ignore matfiles that are in archive files
fun = lambda s: 'archive' not in s.lower()
mat_paths = list(filter(fun, mat_paths))

# Set an amount of frames to drop in front of the traces
front_frame_drop = 14

# Loop through each matfile that contains
not_aligned = []
for mat_i, mat_path in enumerate(mat_paths): #TODO DEBUG mat_paths:    
    print(str(mat_i) +' '+ mat_path)

    #Get the current experiments data 
    cur_flicker = get_param(os.path.basename(mat_path), 'flicker')
    cur_freq = get_param(os.path.basename(mat_path), 'DBS')
    cur_fov = get_param(os.path.basename(mat_path), 'fov')

    # Set the experimental parameters into variables
    cur_stim_freq = int(cur_freq.replace('DBS', '').replace('hz', ''))
    cur_flicker_freq = int(cur_flicker.replace('flicker', '').replace('hz', ''))
    session_id = os.path.basename(os.path.dirname(mat_path))
    mouse_id = os.path.basename(os.path.dirname(os.path.dirname(mat_path)))
    stim_param_string = 'currentamp:' + get_param(os.path.basename(mat_path), 'ua') 

    #Find corresponding experiment openephys data
    ephys_dir = f.join([os.path.dirname(mat_path), 'ephys']) + f
    ephys_path = glob(ephys_dir + '*' + cur_flicker + '*' + \
                        cur_freq + '*' + cur_fov + '*')
    if not ephys_path:
        print('Missing Corresponding Ephys Folder')
        print('Matfile: ' + os.path.basename(mat_path))
        not_aligned = not_aligned.append(mat_path)
        continue

    if len(ephys_path) > 1:
        print('There are multiple ephys_path\'s with this experiment')
        not_aligned = not_aligned.append(mat_path)
        continue

    # Read in all of the Open Ephys data
    #1 are the frames
    #3 are the flicker pulses
    #4 are the stim pulses
    #5 are the trial start triggers
    stim_timestamps, frame_timestamps, flicker_timestamps = \
        read_openephys_data(pathname=ephys_path[0], start_chan='ADC5', frame_chan='ADC1', \
                            stim_chan='ADC4', flicker_chan='ADC3')

    # Read in trace data
    data = read_mat(mat_path)
    sp_info = data['spike_detect_SA_v4_info']
    data = data['roi_list']

    #TODO figure out how to check if there are multiple neuron traces
    if len(data['traces'].shape) < 2:
        data['traces'] = data['traces'].reshape(-1, 1)
    
    for roi_i in range(data['traces'].shape[1]):

        # Loop through each trial
        for trial_i in range(len(sp_info)):
            #TODO need to figure out a way to skip this trial during alignment
            # Maybe try to remove that index ephys information across all of its arrays

            # Delete trial elements that do not have the same amount 
            print(trial_i)
            while len(frame_timestamps[trial_i]) < 1:
                print('Removed trial stuff')
                frame_timestamps.pop(trial_i)
                stim_timestamps.pop(trial_i)
                flicker_timestamps.pop(trial_i)
            
            cur_frame_time = frame_timestamps[trial_i][front_frame_drop:]
            raw_trace = data['traces'][np.where(data['trial_vec'] == trial_i + 1)[0], roi_i]
            raw_trace = raw_trace[front_frame_drop:]

            # Grab the first number set of camera frames as the trace
            cur_frame_time = cur_frame_time[:raw_trace.shape[0]]
            block_start_time = cur_frame_time[0]

            cur_frame_time = cur_frame_time - block_start_time
            cur_stim_time = stim_timestamps[trial_i] - block_start_time
            cur_flicker_time = flicker_timestamps[trial_i] - block_start_time
            bl_range = cur_frame_time[-1] - cur_frame_time[0]

            # Grab the original spike detection data
            cur_spike_raster = np.zeros_like(raw_trace)
            spike_idx = sp_info[trial_i*(data['traces'].shape[1]) + roi_i]['spike_idx']
            if isinstance(spike_idx, int):
                spike_idx = np.array([spike_idx])

            spike_idx = spike_idx[np.where(spike_idx >= front_frame_drop + 1)]

            cur_spike_raster[spike_idx - front_frame_drop - 1] = 1

            # Define the 'interp_time' and interpolate all of the data
            interp_time = np.linspace(0, np.floor(bl_range), raw_trace.shape[0])
            interp_raw_trace = np.interp(interp_time, cur_frame_time, raw_trace)
            interp_spike_raster = np.interp(interp_time, cur_frame_time, cur_spike_raster)

            # Interpolate subthreshold Vm, with spikes removed
            sub_vm = sp_info[trial_i*(data['traces'].shape[1]) + roi_i]['trace_spikeRemoved'][front_frame_drop:]
            interp_subvm = np.interp(interp_time, cur_frame_time, sub_vm)

            # Adjusted spike index
            peak_idx, _ = signal.find_peaks(interp_spike_raster)
            interp_spike_raster_adj = np.zeros(interp_raw_trace.shape)
            interp_spike_raster_adj[peak_idx] = 1

            # Create stimulation raster
            cur_stim_raster = np.zeros(raw_trace.shape[0])
            diff_mat = [interp_time] - np.transpose([cur_stim_time])
            diff_mat[diff_mat < 0] = float('inf')
            stim_idx_i = np.argmin(diff_mat, axis=1)
            cur_stim_raster[stim_idx_i] = 1

            # Create flicker raster
            # Calculate the light duration for 1Hz
            cur_flicker_raster = np.zeros(raw_trace.shape[0])
            diff_mat = [interp_time] - np.transpose([cur_flicker_time])
            diff_mat[diff_mat < 0] = float('inf')
            flicker_idx_i = np.argmin(diff_mat, axis=1)
            cur_flicker_raster[flicker_idx_i] = 1
            flick_dur = 0.06256 # Calculated one time
            flick_pts = flick_dur/np.mean(np.diff(cur_frame_time))

            mask = np.zeros(cur_flicker_raster.shape[0])

            #Construct mask for flicker
            for i in np.where(cur_flicker_raster == 1)[0]:
                mask[np.arange(i, i + flick_pts + 1, dtype=int)] = 1
            mask = mask.astype(bool)
            cur_flicker_raster[mask] = 1

            trace_dict = {'frame_time':cur_frame_time,
                          'raw_trace':raw_trace,
                          'interp_time':interp_time,
                          'interp_raw_trace':interp_raw_trace,
                          'interp_spike_raster':interp_spike_raster_adj,
                          'flicker_raster':cur_flicker_raster,
                          'stim_raster':cur_stim_raster,
                          'interp_subvm':interp_subvm,
                          'stim_freq':cur_stim_freq,
                          'flicker_freq':cur_flicker_freq,
                          'mouse_id':mouse_id,
                          'session_id':session_id,
                          'trial_id':trial_i,
                          'stim_param':stim_param_string,
                          'fov_id':int(cur_fov.replace('fov', '')),
                          'roi_id':roi_i
                          }
            
            df = pd.concat([df, pd.DataFrame(trace_dict)], ignore_index=True, join='outer')

            df['stim_freq'] = df['stim_freq'].astype('category')
            df['mouse_id'] = df['mouse_id'].astype('category')
            df['session_id'] = df['session_id'].astype('category')

# Print the not aligned matfiles
if not not not_aligned:
    print('Not aligned Matfiles')
    for mf in not_aligned:
        print(mf)


In [ ]:
# Save data frame to pickle file
save_filename = f + 'home' +f+ 'pierfier' +f+ 'Projects' +f+ 'Pierre Fabris' +f+ 'PV DBS neocortex' +f+ 'Interm_Data' +f+ 'flicker.pkl'


In [ ]:
plt.close('all')

In [ ]:
print(ephys_path)
for path in ephys_path:
    print(path)

In [ ]:
print(len(frame_timestamps))
print(len(flicker_timestamps))
print(len(sp_info))
print(trial_i)

In [ ]:
print(df['mouse_id'].unique())
print(df['fov_id'].unique())
print(df[ (df['fov_id'] == 1) & (df['mouse_id'] == '109558_Vb_male')])